In [1]:
from importlib import reload
import audioDatastore.AudioDatastore as myads
reload(myads)
import os.path
from collections import Counter
import random
from python_speech_features import mfcc
import scipy.io.wavfile as wav
from spafe.utils import vis
import matplotlib.pyplot as plt
import misc.vad as vad
reload(vad)

<module 'misc.vad' from 'C:\\Users\\David\\OneDrive\\Desktop\\pythonProjects\\ubm\\misc\\vad.py'>

In [2]:
# [1] separate data into train, enroll and verify

datasetFolder = r"C:\Users\David\OneDrive\Desktop\matlab\data\speech_commands_v0.01"
main_ads = myads.AudioDatastore()
main_ads.populate(datasetFolder,include_sub_folders=True, label_source=True)

In [3]:
ads = myads.subset(main_ads, label='stop')

speakers = []
for file in ads.files:
    nm = os.path.basename(file)
    nm = nm.split('_')[0]
    speakers.append('a' + nm)

ads.set(labels=speakers)

In [4]:
num_speakers_to_enroll = 10
label_count = Counter(ads.labels)
for_enroll_and_test_set = []
for key, cnts in list(label_count.items()):
    if cnts >= 3:
        for_enroll_and_test_set.append(key)
for_enroll = random.sample(for_enroll_and_test_set, num_speakers_to_enroll)
ads_enroll_and_validate = myads.subset(ads, label=for_enroll)
ads_enroll, _ = myads.split(ads_enroll_and_validate, int(len(ads_enroll_and_validate.labels) / 2))
ads_test = myads.subset(ads, for_enroll_and_test_set)
ads_test = myads.filter(ads_test, ads_enroll.labels)
ads_train_ubm = myads.filter(ads, ads_test.labels)
ads_train_ubm = myads.filter(ads_train_ubm, ads_enroll.labels)

In [5]:
# [2] feature extraction
# 	• Normalize the audio
# 	• Use detectSpeech to remove nonspeech regions from the audio
# 	• Extract features from the audio
# 	• Normalize the features
#   * Apply cepstral mean normalization

def helper_feature_extraction(audio_file, norm_factors = None):
    # read in file
    (rate,sig) = wav.read(audio_file)

    # normalise
    sig = sig / max(sig)

    # detect speech => convert this into one function
    v = vad.VoiceActivityDetector(rate, sig, 0.5)
    detected_speach = v.detect_speech()
    idx = v.convert_windows_to_readible_labels(detected_speach)
    if idx:
        sig = sig[idx[0]:idx[1]]
        mfcc_feat = mfcc(sig,rate)

        # feature normalisation and Cepstral mean subtraction (for channel noise)
        if norm_factors:
            mfcc_feat = (mfcc_feat - norm_factors.means) / norm_factors.std
            mfcc_feat = mfcc_feat - np.mean(mfcc_feat)

        return mfcc_feat
    else:
        return []


In [6]:
import pickle

try:
    all_features = pickle.load(open('all_features.pickle','rb'))
except:
    all_features = []
    for file in ads.files:
        feature = helper_feature_extraction(file)
        all_features.append(feature)
    pickle.dump(all_features, open('all_features.pickle','wb'))


In [7]:
# collect normalization factors, why and where is this used ?

import numpy as np
# means = np.mean(all_features.fl, axis=0)
means = []
std = []
for feature in all_features:
    if len(feature) > 0:
        means.append(np.mean(feature, axis=0))
        std.append(np.std(feature, axis=0))

means = np.array(means)
means = np.mean(means, axis=0)

std = np.array(std)
std = np.std(std, axis=0)

class NormFactor:
  def __init__(self, m, s):
    self.means = m
    self.std = s

normFactors = NormFactor(means, std)

print(normFactors.means)

[ -2.90282935  -0.33093011 -20.11337245 -14.39478988 -20.00294044
   1.69950359   2.69763417  -0.9433615   -8.80432994   2.05696167
  -4.85549398  -3.99526123  -9.69053713]


In [8]:
# normalise features, need to re-look at this ? how does cepstral mean normalization work ?

normalised_features = []
for i in range(len(all_features)):
    if len(all_features[i]) > 0:
        normalised = (all_features[i] - means) / std
        normalised = normalised - np.mean(normalised)
        normalised_features.append(normalised)
    else:
        normalised_features.append(np.array([]))


In [83]:
# [3] gmm ubm EM

# 3.a collect features
maxFeatures = 100 # len(ads_train_ubm.files) # len(
train_features = []
for f in range(maxFeatures):
    # extract a feature
    train_features.append(helper_feature_extraction(ads_train_ubm.files[f], normFactors))

In [84]:
# 3.b create UBM using sklearn

from sklearn.mixture import GaussianMixture, _base, _gaussian_mixture

numComponents = 32
alpha = np.ones(numComponents) / numComponents
numFeatures = normalised_features[0].shape[1]
mu = np.random.random((numComponents,numFeatures))
sigma = np.random.random((numComponents,numFeatures))

ubm = GaussianMixture(n_components=numComponents, covariance_type='diag', weights_init=alpha, means_init=mu, precisions_init=sigma)

ubm.means_ = mu
ubm.covariances_ = sigma
ubm.weights_ = alpha
ubm.precisions_cholesky_ = _gaussian_mixture._compute_precision_cholesky(sigma, 'diag')

In [85]:
# 3.c em algorithm
from scipy.special import logsumexp

maxIter = 100
targetLogLikelihood = 0
tol = 1e-3
pastL = -np.inf

for i in range(maxIter):

    # E step
    N = np.zeros((1, numComponents))
    F = np.zeros((numFeatures, numComponents))
    S = np.zeros((numFeatures, numComponents))
    L = 0
    eps = np.finfo(np.float64).eps

    for feature in train_features:
        # check for sub par samples
        if len(feature) > 0:
            logLikelihood = ubm._estimate_weighted_log_prob(feature)
            logLikelihoodSum = logsumexp(logLikelihood, axis=1)
            gamma = np.exp((logLikelihood.T - logLikelihoodSum)).T

            # Compute Baum-Welch stats
            n = sum(gamma,1)
            f = np.dot(feature.T, gamma)
            s = np.dot(feature.T * feature.T, gamma)

            N = N + n
            F = F + f
            S = S + s

            L = L + sum(logLikelihoodSum)

    # M step
    L = L / len(train_features)
    if i%10 == 0:
        print('iter: ' + str(i) + ', log likelihood ' + str(L))
    if L > targetLogLikelihood or np.abs(pastL - L) < tol:
        print('converged on iter: ' + str(i))
        ubm.converged_ = True
        break
    else:
        pastL = L

    N = np.maximum(N, eps)
    weights = np.maximum(N / np.sum(N), eps)
    newWeights = weights / np.sum(weights)  # why again, because of eps ...
    newMeans = F / N
    newCovariances = np.maximum((S / N) - np.square(ubm.means_).T, eps)
    ubm.weights_ = np.squeeze(newWeights.T)
    ubm.means_ = newMeans.T
    ubm.covariances_ = newCovariances.T


iter: 0, log likelihood -5678.877473015325
iter: 10, log likelihood -4421.958998248674
iter: 20, log likelihood -4406.879645726132
iter: 30, log likelihood -4387.929159325731
iter: 40, log likelihood -4386.449779104133
converged on iter: 45


In [82]:
# sklearn does better - probably because of k means init

ubm_test = GaussianMixture(n_components=32, covariance_type='diag')
train_features_flattened = np.array([item for sublist in train_features for item in sublist])
ubm_test.fit(train_features_flattened)
print('sklearn model' + str(ubm_test.score(train_features[0])))
print('my model' + str(ubm.score(train_features[0])))

sklearn model-36.98687026025601
my model-232.77429554284976


In [11]:
# enrol



C:\Users\David\miniconda3\envs\ubm\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


-20.475179198022968

In [ ]:

# for i in len(maxIter):

# N = np.zeros(numComponents)
# F = np.zeros((numFeatures, numComponents))
# S = np.zeros((numFeatures, numComponents))
# L = 0
# feature = 0
#
# for i in range(maxIter):
#     for f in range(maxFeatures):
#         # extract a feature
#         feature = helper_feature_extraction(ads_train_ubm.files[f], normFactors)
#
#         # compute a posteriori log-likelihood
#         logLikelihood = helperGMMLogLikelihood(feature,ubm)
